<a href="https://colab.research.google.com/github/20021456/mycode/blob/main/NLP2SQLwith2Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Môi trường cài đặt

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain-experimental

In [ ]:
!pip install -q langchain
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install  google-search-results -q

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install  langchain openai pymysql --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.6 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
from langchain import OpenAI, SQLDatabase

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-fWh3h6eO1l8amh8imvHuT3BlbkFJyFtKnhLY5MDhGIGAoKHf'

# Đầu vào

In [ ]:
import numpy as np
import pandas as pd
import sqlite3

df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP to SQL/Telco_customer_churn_services.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP to SQL/Telco_customer_churn_status.csv")

conn = sqlite3.connect('memory.sqlite')
df1.to_sql('services', conn, if_exists='replace', index=False)
df2.to_sql('customers', conn, if_exists='replace', index=False)

7043

In [ ]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [ ]:
import sqlite3
read_sql_query('SELECT * FROM customers LIMIT 10;',
               "memory.sqlite")

('8779-QRDMV', 1, 'Q3', 3, 'Churned', 'Yes', 1, 91, 5433, 'Competitor', 'Competitor offered more data')
('7495-OOKFY', 1, 'Q3', 3, 'Churned', 'Yes', 1, 69, 5302, 'Competitor', 'Competitor made better offer')
('1658-BYGOY', 1, 'Q3', 2, 'Churned', 'Yes', 1, 81, 3179, 'Competitor', 'Competitor made better offer')
('4598-XLKNJ', 1, 'Q3', 2, 'Churned', 'Yes', 1, 88, 5337, 'Dissatisfaction', 'Limited range of services')
('4846-WHAFZ', 1, 'Q3', 2, 'Churned', 'Yes', 1, 67, 2793, 'Price', 'Extra data charges')
('4412-YLTKF', 1, 'Q3', 1, 'Churned', 'Yes', 1, 95, 4638, 'Competitor', 'Competitor had better devices')
('0390-DCFDQ', 1, 'Q3', 2, 'Churned', 'Yes', 1, 76, 3964, 'Other', "Don't know")
('3445-HXXGF', 1, 'Q3', 1, 'Churned', 'Yes', 1, 91, 5444, 'Dissatisfaction', 'Service dissatisfaction')
('2656-FMOKZ', 1, 'Q3', 2, 'Churned', 'Yes', 1, 91, 5717, 'Dissatisfaction', 'Limited range of services')
('2070-FNEXE', 1, 'Q3', 2, 'Churned', 'Yes', 1, 81, 4419, 'Price', 'Lack of affordable download/u

In [ ]:
llm_1 = OpenAI(temperature=0)
input_db = SQLDatabase.from_uri('sqlite:///memory.sqlite')

In [ ]:
db_agent = SQLDatabaseChain(llm = llm_1,
                            database = input_db,
                            verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


#Question 1

In [ ]:
chain = db_agent.run("how many rows are there?")
print(chain)



> Entering new SQLDatabaseChain chain...
how many rows are there?
SQLQuery:SELECT COUNT(*) FROM customers
SQLResult: [(7043,)]
Answer:7043
> Finished chain.
7043


In [ ]:
chain = db_agent.run("how many rows are there?")
for step in chain.steps:
  print(step.get_query())



> Entering new SQLDatabaseChain chain...
how many rows are there?
SQLQuery:SELECT COUNT(*) FROM customers
SQLResult: [(7043,)]
Answer:7043
> Finished chain.


AttributeError: 'str' object has no attribute 'steps'

In [ ]:
db_agent.run("Tìm ID và giá trị tổng phí của 5 người có tổng doanh thu cao nhất")



> Entering new SQLDatabaseChain chain...
Tìm ID và giá trị tổng phí của 5 người có tổng doanh thu cao nhất
SQLQuery:SELECT "Customer ID", "Total Charges" FROM memory_products ORDER BY "Total Revenue" DESC LIMIT 5
SQLResult: [('0164-APGRB', 8496.7), ('8263-QMNTJ', 8425.3), ('5451-YHYPW', 8443.7), ('3810-DVDQQ', 8308.9), ('7569-NMZYQ', 8672.45)]
Answer:5 người có tổng doanh thu cao nhất là:
- ID: 0164-APGRB, Tổng phí: 8496.7
- ID: 8263-QMNTJ, Tổng phí: 8425.3
- ID: 5451-YHYPW, Tổng phí: 8443.7
- ID: 3810-DVDQQ, Tổng phí: 8308.9
- ID: 7569-NMZYQ, Tổng phí: 8672.45
> Finished chain.


'5 người có tổng doanh thu cao nhất là:\n- ID: 0164-APGRB, Tổng phí: 8496.7\n- ID: 8263-QMNTJ, Tổng phí: 8425.3\n- ID: 5451-YHYPW, Tổng phí: 8443.7\n- ID: 3810-DVDQQ, Tổng phí: 8308.9\n- ID: 7569-NMZYQ, Tổng phí: 8672.45'

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import re
from logging import PlaceHolder
chat_history = []

def sql_query(df, query):
    try:
        result_df = pd.read_sql_query(query, conn)
        return result_df
    except:
        return "Incorrect SQL Query. Try again"

def on_submit(_):
  query = input_box.value
  input_box.value = ""

  if query.lower() == "exit":
    print("Thank you for using the sql chatbot")
    return

  result = db_agent.run(query)

  if not result:
    # Nếu không tìm thấy thông tin, có thể sử dụng ChatGPT để tìm kiếm
    print(result)
    return

  chat_history.append((query, result))

  display(widgets.HTML(f'<b>User: </b> {query}'))
  display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result}'))
print("Welcome to the sql Chatbot! Type 'exit' to stop.")

input_box = widgets.Text(PlaceHolder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Welcome to the sql Chatbot! Type 'exit' to stop.


Text(value='')



> Entering new SQLDatabaseChain chain...
Tổng doanh thu của tất cả khách
SQLQuery:SELECT SUM("Total Revenue") FROM services
SQLResult: [(21371131.690000027,)]
Answer:21371131.690000027
> Finished chain.


HTML(value='<b>User: </b> Tổng doanh thu của tất cả khách')

HTML(value='<b><font color="blue">Chatbot:</font></b> 21371131.690000027')

...

HTML(value='<b>User: </b> Điểm trung bình của độ hài lòng lifetime value là bao nhiêu')

HTML(value='<b><font color="blue">Chatbot:</font></b> 3.2449240380519666')



> Entering new SQLDatabaseChain chain...
Điểm trung bình của satisfaction lifetime value của các khách hàng rời bỏ là bao nhiêu
SQLQuery:SELECT AVG("CLTV") FROM customers WHERE "Customer Status" = 'Churned'
SQLResult: [(4149.414660246121,)]
Answer:4149.414660246121
> Finished chain.


HTML(value='<b>User: </b> Điểm trung bình của satisfaction lifetime value của các khách hàng rời bỏ là bao nhi…

HTML(value='<b><font color="blue">Chatbot:</font></b> 4149.414660246121')